In [107]:
import glob, os
import shutil
import json
import random

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [92]:
num_workers = 3
dataset = 'kermany'
dataset_info = 'kermany_metadata.csv'
worker_path = '.'

metadata = {
    "datatype": "image",
    "operations": {
        'use_grayscale': True,
        'use_alpha': False,
        'use_deepaugment': True
    }
}

In [93]:
for i in range(num_workers):
    os.makedirs(worker_path + '/worker_{0}/{1}/train'.format(i, dataset))
    os.makedirs(worker_path + '/worker_{0}/{1}/test'.format(i, dataset))
    # create metadata.json
    with open(worker_path + '/worker_{0}/{1}/train/metadata.json'.format(i, dataset), 'w') as f:
        json.dump(metadata, f)
    with open(worker_path + '/worker_{0}/{1}/test/metadata.json'.format(i, dataset), 'w') as f:
        json.dump(metadata, f)

In [94]:
df = pd.read_csv('kermany_metadata.csv')
df.head()

,path,target,subject,serial,image,source
0,./train/NORMAL/NORMAL-4161487-6.jpeg,0,4161487,6,NORMAL-4161487-6.jpeg,train
1,./train/NORMAL/NORMAL-9103640-1.jpeg,0,9103640,1,NORMAL-9103640-1.jpeg,train
2,./train/NORMAL/NORMAL-9593905-3.jpeg,0,9593905,3,NORMAL-9593905-3.jpeg,train
3,./train/NORMAL/NORMAL-8115192-10.jpeg,0,8115192,10,NORMAL-8115192-10.jpeg,train
4,./train/NORMAL/NORMAL-1498414-1.jpeg,0,1498414,1,NORMAL-1498414-1.jpeg,train


In [106]:
df[df['source'] == 'train']['subject'].unique()

array([4161487, 9103640, 9593905, ..., 8743785, 4228560, 9983517])

In [127]:
unique_train_subjects = df[df['source'] == 'train']['subject'].unique()
unique_test_subjects = df[df['source'] == 'test']['subject'].unique()
random.Random(42).shuffle(unique_train_subjects)
random.Random(42).shuffle(unique_test_subjects)
train_workers = np.array_split(unique_train_subjects, num_workers)
test_workers = np.array_split(unique_test_subjects, num_workers)
for i in range(num_workers):
    print("worker_" + str(i), len(train_workers[i]), len(test_workers[i]))

worker_0 1595 212
worker_1 1595 212
worker_2 1595 211


In [128]:
for i in range(num_workers):
    # create mapping.csv
    df[df.subject.isin(train_workers[i])][['image', 'target']].to_csv(worker_path + '/worker_{0}/{1}/train/mapping.csv'.format(i, dataset), index=False)
    df[df.subject.isin(test_workers[i])][['image', 'target']].to_csv(worker_path + '/worker_{0}/{1}/test/mapping.csv'.format(i, dataset), index=False)
    
    # distribute the images
    for file in df[df.subject.isin(train_workers[i])].path.values:
        shutil.copy2(file, worker_path + "/worker_{0}/{1}/train/".format(i, dataset))
    print('{1} train data for worker_{0} is transferred'.format(i, dataset))
    for file in df[df.subject.isin(test_workers[i])].path.values:
        shutil.copy2(file, worker_path + "/worker_{0}/{1}/test/".format(i, dataset))
    print('{1} test data for worker_{0} is transferred'.format(i, dataset))

kermany train data for worker_0 is transferred
kermany test data for worker_0 is transferred
kermany train data for worker_1 is transferred
kermany test data for worker_1 is transferred
kermany train data for worker_2 is transferred
kermany test data for worker_2 is transferred
